In [9]:
import json
import pandas as pd
import collections
import re


with open(r"data_pdfs.json") as f:
    base_data = json.load(f)

save_pdfs = []
lista_de_preços_iniciais = []
lista_de_preços_efetivos = []
lista_de_ids = []
lista_de_descrições = []
Label_ID = []
lista_de_prazo_de_execução = []
lista_local_de_execução = []
lista_de_prazos_de_submissão = []
lista_de_distritos = []
lista_de_datas_de_publicação = []
lista_de_datas_de_fecho = []
lista_de_c_ambientais = []
lista_de_datas_de_assinatura = []
lista_de_end_of_contracts = []
lista_de_numero_de_contestants = []
lista_de_justificações_prazo = []
lista_de_justificações_preço = []
lista_de_cpvs = []
lista_de_cpvs_designations = []
lista_de_preços_base = []
lista_de_valores_de_caução = []
Jornal_da_EU = []
info_adicional = []
lista_de_concelhos = []

for alinea in base_data:
    informação = alinea["extra_details"]
    Objecto_do_contrato = alinea["objectBriefDescription"] 
    preço_contratual = alinea["initialContractualPrice"]
    ID_do_contrato = alinea["id"]
    lista_de_preços_iniciais.append(preço_contratual)
    lista_de_ids.append(ID_do_contrato)
    lista_de_descrições.append(Objecto_do_contrato)

    if informação == None:
        preço_efectivo = "Anuncio sem detalhes extra"
        lista_de_preços_efetivos.append(preço_efectivo)
        lista_de_datas_de_fecho.append("")
        lista_de_distritos.append("")
        lista_de_concelhos.append("")
        lista_local_de_execução.append("")
        lista_de_prazo_de_execução.append("")
        lista_de_datas_de_assinatura.append("")
        lista_de_end_of_contracts.append("")
        lista_de_numero_de_contestants.append("")
        lista_de_justificações_prazo.append("")
        lista_de_justificações_preço.append("")
        lista_de_cpvs.append("")
        lista_de_cpvs_designations.append("")
        lista_de_c_ambientais.append("")
        
    else:
        preço_efectivo = informação["totalEffectivePrice"]
        lista_de_preços_efetivos.append(preço_efectivo)
        prazo_de_execução = informação["executionDeadline"]
        lista_de_prazo_de_execução.append(prazo_de_execução)
        local_de_execução = informação["executionPlace"]
        distrito = local_de_execução.split(',')
        middleIndex = int((len(distrito) - 1)/2)
        lista_local_de_execução.append(local_de_execução)
        lista_de_distritos.append(distrito[middleIndex])
        lista_de_concelhos.append(distrito[-1])
        data_de_fecho = informação["closeDate"]
        lista_de_datas_de_fecho.append(data_de_fecho)
        criterio_ambiental = informação["ambientCriteria"]
        lista_de_c_ambientais.append(criterio_ambiental)
        data_de_assinatura = informação["signingDate"]
        lista_de_datas_de_assinatura.append(data_de_assinatura)
        lista_de_end_of_contracts.append(informação["endOfContractType"])
        lista_de_numero_de_contestants.append(len(informação["contestants"]))
        lista_de_justificações_prazo.append(informação["causesDeadlineChange"])
        lista_de_justificações_preço.append(informação["causesPriceChange"])
        lista_de_cpvs.append(informação["cpvs"])
        lista_de_cpvs_designations.append(informação["cpvsDesignation"])
        



# get criterio de adj
for alinea in base_data:
    tamanho_antes = len(lista_de_valores_de_caução)
    if "anuncio_details" in alinea:
        detalhes = alinea["anuncio_details"]
        prazo_de_submissão = detalhes["proposalDeadline"]
        lista_de_prazos_de_submissão.append(prazo_de_submissão)
        prazo_de_publicação = detalhes["drPublicationDate"]
        lista_de_datas_de_publicação.append(prazo_de_publicação[-4:])
        lista_de_preços_base.append(detalhes["basePrice"])
        
        if detalhes["type"] in ["Aviso de prorrogação de prazo", "Declaração de retificação de anúncio"]:
            save_pdfs.append(detalhes["type"])
            Label_ID.append(0)
            lista_de_valores_de_caução.append("")
            Jornal_da_EU.append("")
            info_adicional.append("")
        elif "pdf_anuncio" in detalhes:
            anuncios = detalhes["pdf_anuncio"]
            encontrado = False
            encontrado_prest = False
            encontrado_EU = False
            encontrado_infoadd = False
            for anuncio in anuncios:
                dados_do_pdf = anuncio["data"]
                for campos in dados_do_pdf:
                    crit = campos["fieldDESC"]
                    if crit in ["INDICAÇÕES ADICIONAIS"]:
                        info_adicional.append(campos["fieldTEXT"])
                        encontrado_infoadd = True
                    if crit in ["O PROCEDIMENTO A QUE ESTE ANÚNCIO DIZ RESPEITO TAMBÉM É PUBLICITADO NO JORNAL OFICIAL DA", "O PROCEDIMENTO A QUE ESTE ANÚNCIO DIZ RESPEITO TAMBÉM É"]:
                        Jornal_da_EU.append(campos["fieldTEXT"])
                        encontrado_EU = True
                    if crit in ["PRESTAÇÃO DE CAUÇÃO", "PRESTAÇÃO DE CAUÇÃO:", "PRESTAÇÃO DE CAUÇÃO: ", "DISPENSA DE PRESTAÇÃO DE CAUÇÃO:  Não", "DISPENSA DE PRESTAÇÃO DE CAUÇÃO:  Sim"]:
                        caução = campos["fieldTEXT"]
                        lista_de_valores_de_caução.append(crit[-3:])
                        encontrado_prest = True
                    elif crit in ["Dispensa de prestação de caução"]:
                        lista_de_valores_de_caução.append(crit)
                        encontrado_prest = True
                    if crit in  ['CRITÉRIO DE ADJUDICAÇÃO', "Critério de adjudicação (*)", "Critério de adjudicação (*) "]:
                        criterio = campos["fieldTEXT"]
                        save_pdfs.append(criterio)
                        encontrado = True
                        if "100" in criterio:
                            Label_ID.append(1)
                        elif "Preço\n \n" == criterio:
                            Label_ID.append(1)
                        elif re.search(" 0", criterio):
                            Label_ID.append(1)
                        elif "Mais baixo preço" in criterio:
                            Label_ID.append(1)
                        elif "proposta economicamente mais vantajosa\nfatores e eventuais subfatores acompanhados dos respetivos coeficientes de ponderação: de acordo com " in criterio.lower():
                            Label_ID.append(0)
                        elif "Se o critério for o da proposta economicamente mais vantajosa, indicar os fatores e eventuais \nsubfactores acompanhados dos respetivos coeficientes de ponderação (*)\nClique aqui para introduzir texto.\n" in criterio:
                            Label_ID.append(0)
                        elif "proposta economicamente mais vantajosa\nfatores e eventuais subfatores acompanhados dos respetivos coeficientes de ponderação: remet" in criterio.lower():
                            Label_ID.append(0)
                        elif "proposta economicamente mais vantajosa\nfatores e eventuais subfatores acompanhados dos respetivos coeficientes de ponderação: conforme" in criterio.lower():
                            Label_ID.append(0)
                        elif "proposta economicamente mais vantajosa\nfatores e eventuais subfatores acompanhados dos respetivos coeficientes de ponderação: de acordo " in criterio.lower():
                            Label_ID.append(0)
                        else:
                            Label_ID.append(2) 
            if not encontrado:
                save_pdfs.append(detalhes["type"] + " sem Pdf")
                Label_ID.append(0)
            if not encontrado_prest:
                lista_de_valores_de_caução.append("")
            if not encontrado_EU:
                Jornal_da_EU.append("")
            if not encontrado_infoadd:
                info_adicional.append("")
        else: 
            save_pdfs.append("Link do anúncio com hiperligação errada \ falha a baixar pdf")
            Label_ID.append(0)
            lista_de_valores_de_caução.append("")
            Jornal_da_EU.append("")
            info_adicional.append("")
    else: 
        save_pdfs.append("Contrato sem detalhes acerca do anúncio")
        Label_ID.append(0)
        lista_de_prazos_de_submissão.append("")
        lista_de_datas_de_publicação.append("")
        lista_de_preços_base.append("")
        lista_de_valores_de_caução.append("")
        tamanho_depois = len(lista_de_valores_de_caução)
        Jornal_da_EU.append("")
        info_adicional.append("")
    

print(type(criterio))
# table
lista_de_critérios = pd.Series(save_pdfs)
tabela = pd.DataFrame(columns=['ID', "Brief Discription", "Cpvs", "Cpvs Designation", "Ambient Criteria (T/F)", "Publication Year", "Signing Date", "Close Date", 'Location', "Municipality", "District", "Base Price", 'Initial Price','Effective Price', "Submission Deadline", "Execution Deadline", 'Critério de adjudicação', "Label ID", "Caução", "Published in EU Journal", "End of Contract Type", "Number of Contestants", "Deadline Change Justification", "Price Change Justification"])
tabela['ID'], tabela["Brief Discription"], tabela["Publication Year"], tabela["Close Date"], tabela["Location"], tabela["District"], tabela['Initial Price'], tabela['Effective Price'], tabela["Execution Deadline"], tabela["Submission Deadline"], tabela['Critério de adjudicação'], tabela["Label ID"]= lista_de_ids, lista_de_descrições, lista_de_datas_de_publicação, lista_de_datas_de_fecho, lista_local_de_execução, lista_de_distritos, lista_de_preços_iniciais, lista_de_preços_efetivos, lista_de_prazo_de_execução, lista_de_prazos_de_submissão, lista_de_critérios, Label_ID
tabela["Ambient Criteria (T/F)"], tabela["Signing Date"], tabela["End of Contract Type"], tabela["Number of Contestants"], tabela["Deadline Change Justification"], tabela["Price Change Justification"] = lista_de_c_ambientais, lista_de_datas_de_assinatura, lista_de_end_of_contracts, lista_de_numero_de_contestants, lista_de_justificações_prazo, lista_de_justificações_preço
tabela["Cpvs"], tabela["Cpvs Designation"], tabela["Base Price"], tabela["Caução"], tabela["Published in EU Journal"], tabela["Municipality"] = lista_de_cpvs, lista_de_cpvs_designations, lista_de_preços_base, lista_de_valores_de_caução, Jornal_da_EU, lista_de_concelhos

 
tabela_excel = "table_.xlsx"
tabela.to_excel(tabela_excel)

print(tabela.head())
print(len(lista_de_critérios))
print(len(base_data))

<class 'str'>
        ID                                  Brief Discription        Cpvs  \
0  6855032  Rede de Drenagem de Águas Residuais e Abasteci...  45231300-8   
1  5920676  Plano de Salvaguarda do Vale do Bestança - Con...  45200000-9   
2  5908387  Requalificação das Ruas "Capitão Salgueiro Mai...  45211360-0   
3  4858933  Empreitada de Obras públicas para a Remodelaçã...  45453000-7   
4  4558120  5010015061 - Empreitada de "Linha do Norte, Su...  45233261-6   

                                    Cpvs Designation  Ambient Criteria (T/F)  \
0  Construção de condutas para água e águas resid...                   False   
1  Obras de construção total ou parcial e de enge...                   False   
2                               Obras de urbanização                   False   
3                     Obras de revisão e recuperação                   False   
4         Construção de passagem superior para peões                   False   

  Publication Year Signing Date  Close Dat

In [8]:
from ssl import cert_time_to_seconds


len(info_adicional)

envolve_aquisição_conjunta = []
contrato_adj_por_central_de_compras = []
acordo_quadro = []
leilão_electrónico = []
fase_de_negociação = []
contratação_lotes = []

for i in info_adicional:
    alineas_da_info = i.split("\n")
    aquisição = False
    central = False
    quadro = False
    leilao = False
    fase = False
    lotes = False
    for alinea_da_info in alineas_da_info:
        if "aquisição conjunta" in alinea_da_info:
            envolve_aquisição_conjunta.append(alinea_da_info[-3:])
            aquisição = True
        if "central de compras" in alinea_da_info:
            contrato_adj_por_central_de_compras.append(alinea_da_info[-3:])
            central = True
        if "acordo quadro" in alinea_da_info:
            acordo_quadro.append(alinea_da_info[-3:])
            quadro = True
        if "leilão eletrónico" in alinea_da_info:
            leilão_electrónico.append(alinea_da_info[-3:])
            leilao = True
        if "fase de negociação" in alinea_da_info:
            fase_de_negociação.append(alinea_da_info[-3:])
            fase = True
        if "por lotes" in alinea_da_info:
            contratação_lotes.append(alinea_da_info[-3:])
            lotes = True
    if not aquisição:
        envolve_aquisição_conjunta.append("")
    if not central:
        contrato_adj_por_central_de_compras.append("")
    if not quadro:
        acordo_quadro.append("")
    if not leilao:
        leilão_electrónico.append("")
    if not fase:
        fase_de_negociação.append("")
    if not lotes:
        contratação_lotes.append("")
    

tabela2 = pd.DataFrame(columns=["Envolve aquisição conjunta (com várias entidades)?", "Adjudicado por uma central de compras?", "Celebração de um acordo quadro:", "Leilão eletrónico:", "Adotada uma fase de negociação", "Contratação por lotes"])

tabela2["Envolve aquisição conjunta (com várias entidades)?"] = envolve_aquisição_conjunta
tabela2["Adjudicado por uma central de compras?"] = contrato_adj_por_central_de_compras
tabela2["Celebração de um acordo quadro:"] = acordo_quadro
tabela2["Leilão eletrónico:"] = leilão_electrónico
tabela2["Adotada uma fase de negociação"] = fase_de_negociação
tabela2["Contratação por lotes"] = contratação_lotes

tabela2.head()
tabela2_excel = "tabela_info_adicional_2015_v3.xlsx"
tabela2.to_excel(tabela2_excel)


In [2]:
import json
with open(r"C:\Users\up201604212.CAMPUS\Desktop\analise estatistica\Data\Data_de_fecho\base_gov_data_2015_data_de_fecho.json") as f1:
    base_data1 = json.load(f1)
with open(r"C:\Users\up201604212.CAMPUS\Desktop\analise estatistica\Data\Data_de_fecho\base_gov_data_2016_data_de_fecho.json") as f2:
    base_data2 = json.load(f2)
with open(r"C:\Users\up201604212.CAMPUS\Desktop\analise estatistica\Data\Data_de_fecho\base_gov_data_2017_data_de_fecho.json") as f3:
    base_data3 = json.load(f3)
with open(r"C:\Users\up201604212.CAMPUS\Desktop\analise estatistica\Data\Data_de_fecho\base_gov_data_data_de_fecho_2021_2018.json") as f4:
    base_data4 = json.load(f4)
with open(r"C:\Users\up201604212.CAMPUS\Desktop\analise estatistica\Data\Data_de_fecho\base_gov_data_2022_data_de_fecho.json") as f5:
    base_data5 = json.load(f5)

all_data = base_data1 + base_data2 + base_data3 + base_data4 + base_data5

dados_2015_2022 = json.dumps(all_data)
with open("dados_2015_2022.json", "w") as outfile:
    outfile.write(dados_2015_2022)

In [1]:
import json
import pandas as pd 

with open(r"dados_2015_2022.json") as f:
    base_data = json.load(f)

save_pdfs = []
lista_de_preços_iniciais = []
lista_de_preços_efetivos = []
lista_de_ids = []
lista_de_descrições = []
lista_de_prazo_de_execução = []
lista_de_prazos_de_submissão = []


# loop thru contracts 
for alinea in base_data:
    informação = alinea["extra_details"]
    Objecto_do_contrato = alinea["objectBriefDescription"] 
    preço_contratual = alinea["initialContractualPrice"]
    ID_do_contrato = alinea["id"]
    lista_de_preços_iniciais.append(preço_contratual)
    lista_de_ids.append(ID_do_contrato)
    lista_de_descrições.append(Objecto_do_contrato)

#get extra details
    if informação == None:
        preço_efectivo = "Anuncio sem detalhes extra"
        lista_de_preços_efetivos.append(preço_efectivo)
        lista_de_prazo_de_execução.append("N/A")

    else:
        preço_efectivo = informação["totalEffectivePrice"]
        lista_de_preços_efetivos.append(preço_efectivo)
        prazo_de_execução = informação["executionDeadline"]
        lista_de_prazo_de_execução.append(prazo_de_execução)

# get criterio de adj
for alinea in base_data:
    if "anuncio_details" in alinea:
        detalhes = alinea["anuncio_details"]
        prazo_de_submissão = detalhes["proposalDeadline"]
        lista_de_prazos_de_submissão.append(prazo_de_submissão)
    
        if detalhes["type"] in ["Aviso de prorrogação de prazo", "Declaração de retificação de anúncio"]:
            save_pdfs.append(detalhes["type"])
        elif "pdf_anuncio" in detalhes:
            anuncios = detalhes["pdf_anuncio"]
            encontrado = False
            for anuncio in anuncios:
                dados_do_pdf = anuncio["data"]
                for campos in dados_do_pdf:
                    crit = campos["fieldDESC"]
                    if crit in  ['CRITÉRIO DE ADJUDICAÇÃO', "Critério de adjudicação (*)", "Critério de adjudicação (*) "]:
                        criterio = campos["fieldTEXT"]
                        save_pdfs.append(criterio)
                        encontrado = True
            if not encontrado:
                save_pdfs.append(detalhes["type"] + " sem Pdf")

        else: 
            save_pdfs.append("Link do anúncio com hiperligação errada \ falha a baixar pdf")
            
    else: 
        save_pdfs.append("Contrato sem detalhes acerca do anúncio")
        lista_de_prazos_de_submissão.append("N/A")
# create excel table
lista_de_critérios = pd.Series(save_pdfs)
tabela = pd.DataFrame(columns=['ID', "Brief Discription", 'Initial Price','Effective Price', "Execution Deadline", "Submission Deadline" 'Critério de adjudicação'])
tabela['ID'], tabela["Brief Discription"], tabela['Initial Price'], tabela['Effective Price'], tabela["Execution Deadline"], tabela["Submission Deadline"], tabela['Critério de adjudicação'] = lista_de_ids, lista_de_descrições, lista_de_preços_iniciais, lista_de_preços_efetivos, lista_de_prazo_de_execução, lista_de_prazos_de_submissão, lista_de_critérios, Label_ID
tabela_excel = "tabela_2015_2022.xlsx"
tabela.to_excel(tabela_excel)
print(tabela.head())



NameError: name 'save_pdfs' is not defined